In [3]:
import pandas as pd
from umap import UMAP
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
random_state=43

# sns.set_theme(context="paper", style="ticks")

Load data

In [4]:
test = pd.read_csv("data\\test.csv")
train = pd.read_csv("data\\train.csv")

train_X = train.drop("label", axis=1)
train_y = train["label"]

Embed data using umap

In [5]:
reducer = UMAP(
    local_connectivity=1,
    n_neighbors=30,
    n_components=10
)

embedding_train = reducer.fit_transform(
    X=train_X,
    y=train_y
)

embedding_test = reducer.transform(test)

Train Random Forest classifier

In [6]:
nb = RandomForestClassifier()

pred = nb.fit(
    X=embedding_train,
    y=train_y
    ).predict(embedding_test)


save predictions

In [7]:
# 97% Accuracy

submission = pd.DataFrame(columns=["ImageId", "Label"], data=zip(test.index+1, pred))
submission.to_csv("submission.csv", index=False)

# Outlier Detection

In [8]:
# Classification
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV


In [9]:
estimators = [
    ("knnc", KNeighborsClassifier(
        weights="distance",
        n_neighbors=50,
        n_jobs=-1
    )),
    ("gpc", GaussianProcessClassifier(
        random_state=random_state,
        warm_start=True
    )),
    ("rfc", RandomForestClassifier(
        random_state=random_state,
        class_weight="balanced_subsample",
        n_jobs=-1
    )),
    ("SVC", LinearSVC(
        max_iter=100000,
        random_state=random_state
    ))
]

final_estimator = LogisticRegressionCV(
    max_iter=1000,
    # solver="liblinear"
)


In [10]:
reducer = UMAP(
    n_neighbors=30,
    n_components=5,
    min_dist=0,
    # metric="correlation",
    unique=True,
    target_metric="l1",
    target_weight=0.5
)

class_comp = StackingClassifier(
    estimators=estimators,
    # final_estimator=final_estimator,
    n_jobs=-1,
    verbose=0,
)

In [11]:
class_comp.fit(X_train)

NameError: name 'X_train' is not defined